In [1]:
import syft as sy

In [2]:
node = sy.orchestra.launch(name="test-domain-helm2", dev_mode=True, reset=True, n_consumers=4,
                           create_producer=True)
client = node.login(email="info@openmined.org", password="changethis")

INITIALIZING CONSUMER
ABCDEF
INITIALIZING CONSUMER
ABCDEF
INITIALIZING CONSUMER
ABCDEF
INITIALIZING CONSUMER
ABCDEF
Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [3]:
# Question 1: What type of container do we want for data already on the server?

obj_1 = sy.ActionObject.from_obj(1)
ptr_1 = obj_1.send(client)

obj_2 = sy.ActionObject.from_obj(2)
ptr_2 = obj_2.send(client)

obj_3 = sy.ActionObject.from_obj(3)
ptr_3 = obj_3.send(client)

# Option 1: ActionObjects inside ActionObjects
# 
# Pros: very versatile, could work with data from other domains out of the box
# Cons: might not feel intuitive to the user, will need to change the way we work with
#       ActionObjects in a lot of different places in the codebase
# list = sy.ActionObject.from_obj([ptr_1, ptr_2, ptr_3])
# list_ptr = list.send(client)

# Option 2: Create new ActionObjects from the same data
# Will require us to do some value based verification on different objects
# 
# Pros: Easier abstraction for the user
# Cons: Value based verification sounds like an attack vector
#       as it can provide a free Oracle to an attacker
# list = sy.ActionObject.from_list([ptr_1, ptr_2, ptr_3]) # on the server will do ActionObject.from_obj([1,2,3])
# list_ptr = list.send(client)
 

In [4]:
list = sy.ActionObject.from_obj([1,2,3])
list_ptr = list.send(client)

In [5]:
from syft.service.policy.policy import OutputPolicyExecuteCount

# Question 2: What should the UX be for ExecuteOncePerCombination?
# 
# Right now I have worked on using the first option from the previous question
# and using on the fly created lists. We can break this question into more specific ones:
#
# Sub-Question 1: What should we pass for each argument? Should the list be already on the server?
#                 Or can it be defined by the data scientist? 
#                 Could it be made of data outside the domain?
#
# Sub-Question 2: Will anything change if instead of data we talk about files?
#                 The final use case actually will iterate for SyftFiles, so can this affect the UX?
#

@sy.syft_function(input_policy=sy.ExecuteOncePerCombination(
                                    x=list,
                                    y=list,
                                    z=list,
                                ),
                  output_policy=OutputPolicyExecuteCount(limit=27))
def func(x, y, z):
    return x, y, z

SyftSuccess: Syft function 'func' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [6]:
request = client.code.submit(func)
# client.code.request_code_execution(func)
# client.requests[-1].approve()

In [7]:
@sy.syft_function_single_use(list=list_ptr)
def main_func(domain, list):
    jobs = []
    print("start")
    domain.init_checkpoint(27)
    for x in list:
        for y in list:
            for z in list:
                print(x,y,z)
                domain.checkpoint()
                batch_job = domain.launch_job(func, x=x, y=y, z=z)
                jobs.append(batch_job)
                    
    print("done")
    
    return None

SyftSuccess: Syft function 'main_func' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [8]:
client.code.request_code_execution(main_func)
client.requests[-1].approve()

Request approved for domain test-domain-helm2


SyftSuccess: Request d2b5c66398d843e2a094689fb198f978 changes applied

In [9]:
type(client.code[0].input_policy)

NoneType

In [10]:
job = client.code.main_func(list=list_ptr, blocking=False)

In [11]:
job

```python
class Job:
    id: UID = 17244399f3b54cec85824a6aa8f64198
    status: created
    has_parent: False
    result: None
    logs:

0 
    
```

In [15]:
jobs = client.jobs
jobs

LAUNCHING JOB func


FUNCTION LOG ((<UID: 17244399f3b54cec85824a6aa8f64198>, 10)): 1 3 3


LAUNCHING JOB func


FUNCTION LOG ((<UID: 17244399f3b54cec85824a6aa8f64198>, 10)): 2 1 1


LAUNCHING JOB func


...

LAUNCHING JOB func


FUNCTION LOG ((<UID: 17244399f3b54cec85824a6aa8f64198>, 10)): 2 2 1


LAUNCHING JOB func


FUNCTION LOG ((<UID: 17244399f3b54cec85824a6aa8f64198>, 10)): 2 2 2


LAUNCHING JOB func


In [13]:
job_tree = {}
for job in jobs:
    if job.parent_job_id in job_tree:
        job_tree[job.parent_job_id].append(job)
    else:
        job_tree[job.parent_job_id] = [job]

In [14]:
def recursive_print(parent_job, tab_space = 0):
    lines = "─"
    if parent_job.id in job_tree:
        for job in job_tree[parent_job.id]:
            print(f"├─{lines * 2}",  job.id)
            recursive_print(job, tab_space=tab_space+2)

for job in jobs:
    if not job.has_parent:
        print("├─", job.id)
        recursive_print(job, tab_space=2)
        

├─ 17244399f3b54cec85824a6aa8f64198


FUNCTION LOG ((<UID: 17244399f3b54cec85824a6aa8f64198>, 10)): start
FUNCTION LOG ((<UID: 17244399f3b54cec85824a6aa8f64198>, 10)): 1 1 1


LAUNCHING JOB func


FUNCTION LOG ((<UID: 17244399f3b54cec85824a6aa8f64198>, 10)): 1 1 2


LAUNCHING JOB func


...

FUNCTION LOG ((<UID: 17244399f3b54cec85824a6aa8f64198>, 10)): 1 2 3


LAUNCHING JOB func


FUNCTION LOG ((<UID: 17244399f3b54cec85824a6aa8f64198>, 10)): 1 3 1


LAUNCHING JOB func


FUNCTION LOG ((<UID: 17244399f3b54cec85824a6aa8f64198>, 10)): 1 3 2
